# Hyperparameter Tuning using HyperDrive

## Import libraries for Azure Machine Learning SDK

In [1]:
import os
import shutil
import numpy as np 
import pandas as pd 
import azureml.core
import pkg_resources
import joblib
import requests
import json

from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

from azureml.core import Environment, Workspace, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# Check the core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Dataset
### Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()

print('Workspace name:\t\t' + ws.name,
      'Resource group:\t\t' + ws.resource_group,
      'Azure region:\t\t'   + ws.location,
      'Subscription id:\t'  + ws.subscription_id, sep='\n')

Workspace name:		quick-starts-ws-234282
Resource group:		aml-quickstarts-234282
Azure region:		westus2
Subscription id:	d4ad7261-832d-46b2-b093-22156001df5b


### Create an Azure ML experiment

In [3]:
# Name for experiment
experiment_name = 'hyperdrive-heart-failure-experiment'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
hyperdrive-heart-failure-experiment,quick-starts-ws-234282,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute Target
We will need to create a compute target for our AutoML run. We will use ***vm_size = Standard_DS3_v2*** in our provisioning configuration and select ***max_nodes*** to be no greater than 4.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Name for the CPU cluster
amlcompute_cluster_name = "hyperdrive-cpu-compute-cluster"

# Verify that cluster does not exist already
try:
    amlcompute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    amlcompute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', max_nodes=4)
    amlcompute_target = ComputeTarget.create(ws, amlcompute_cluster_name, amlcompute_config)

amlcompute_target.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
compute_targets = ws.compute_targets

for i, key in enumerate(compute_targets):
    print(f"{i+1}. Compute target\n\tname: {compute_targets[key].name}\n\tType: {compute_targets[key].type}")

1. Compute target
	name: notebook234282
	Type: ComputeInstance
2. Compute target
	name: hyperdrive-cpu-compute-cluster
	Type: AmlCompute


In [6]:
# For a more detailed view of current AmlCompute status, use get_status().
print(amlcompute_target.get_status().serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-05-24T15:29:42.530000+00:00', 'errors': None, 'creationTime': '2023-05-24T15:29:37.846701+00:00', 'modifiedTime': '2023-05-24T15:29:44.696952+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The pipeline we use here consists of a custom logistic regression model of the scikit-learn model stored in the ***train.py*** script and a hyperdrive run that iterates through the model parameters.
- Setting logistic regression parameters:
    - --C - Inverse of regularization strenght
    - --max_iter - Maximum number of iterations convergence

***RandomParameterSampling*** defines random sampling over a hyperparameter search space. This sampling algorithm selects parameter values from a series of discrete values or a distribution over a continuous range. This has an advantage over the GridSearch method, which executes all combinations of parameters and takes a long time to execute.

For the "*Inverse of regularization strength*" parameter, I chose a uniform distribution with min=0.0001 and max=1.0. For the maximum number of iterations convergence, I entered a range of values (5, 25, 50, 100, 200, 500, 1000).

***BanditPolicy*** class Defines a early termination policy based on Slack criteria and a frequency and delay interval for evaluation. This goes a long way towards ensuring that if a model with certain parameters does not work well, it is rejected instead of letting it run longer.

The training takes place on the local computing target. Accuracy was chosen as the primary metric with the goal of maximizing accuracy. To keep the training short, max_total_runs was set to 50 and max_duration_minutes to 20.

***max_concurrent_runs = 4*** : The maximum number of iterations that could be run in parallel. It is recommended to create a dedicated cluster per experiment and adjust the number of max_concurrent_iterations of your experiment to the number of nodes in the cluster. In this way you use all nodes of the cluster at the same time with the desired number of concurrent child runs/iterations. So I set the value to 4.

In [12]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
###early_termination_policy = <your policy here>
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
###param_sampling = <your params here>
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.0001, 1.0),
        "--max_iter": choice(5, 25, 50, 100, 200, 500, 1000)
    }
)

script_folder = "./training"

if "training" not in os.listdir():
    os.mkdir(script_folder)
    
shutil.copy('./train.py', script_folder)


'''
SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' 
with your own defined environment or the AzureML-Tutorial curated environment.
'''
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      compute_target=amlcompute_target, 
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=50,   
                                     max_duration_minutes=20,
                                     max_concurrent_runs=4)  

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [13]:
#TODO: Submit your experiment
# Submit the experiment on the configured remote compute cluster
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

In [14]:
# Wait for the cluster to complete, show the output log
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_b818826d-505b-45cb-b217-2efb3f91ff63
Web View: https://ml.azure.com/runs/HD_b818826d-505b-45cb-b217-2efb3f91ff63?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-234282/workspaces/quick-starts-ws-234282&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-05-24T15:29:54.542642][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-05-24T15:29:54.8981986Z][SCHEDULER][INFO]Scheduling job, id='HD_b818826d-505b-45cb-b217-2efb3f91ff63_0' 
[2023-05-24T15:29:55.0209624Z][SCHEDULER][INFO]Scheduling job, id='HD_b818826d-505b-45cb-b217-2efb3f91ff63_1' 
[2023-05-24T15:29:55.1207585Z][SCHEDULER][INFO]Scheduling job, id='HD_b818826d-505b-45cb-b217-2efb3f91ff63_2' 
[2023-05-24T15:29:55.189686][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-05-24T15:29:55.2389175Z][SCHEDULER][INFO]Scheduling job, id='HD_b818826d-505b-45cb-b217-2efb3f91ff

{'runId': 'HD_b818826d-505b-45cb-b217-2efb3f91ff63',
 'target': 'hyperdrive-cpu-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-05-24T15:29:54.102816Z',
 'endTimeUtc': '2023-05-24T15:47:02.091135Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0c2ced38-f2a9-4d73-905a-cc13213180d6',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.8133333333333334',
  'best_child_run_id': 'HD_b818826d-505b-45cb-b217-2efb3f91ff63_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_b818826d-505b-45cb-b217-2efb3f91ff63_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': Non

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

Use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(hyperdrive_run).show()
for children_run in hyperdrive_run.get_children():
    print('-----------------------------------')
    print(children_run)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_b818826d-505b-45cb-b217-2efb3f91ff63_49,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_b818826d-505b-45cb-b217-2efb3f91ff63_48,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_b818826d-505b-45cb-b217-2efb3f91ff63_47,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_b818826d-505b-45cb-b217-2efb3f91ff63_45,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD_b818826d-505b-45cb-b217-2efb3f91ff63_46,
Type: azureml.scriptrun,
Status: Completed)
-----------------------------------
Run(Experiment: hyperdrive-heart-failure-experiment,
Id: HD

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
# evaluate if the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

In [17]:
# Each child in the result has run id, hyperparameters, best primary metric value and status.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# get the best run and display the properties of the model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print("Best Experiment Run:")
print("-------------------------------------------")
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('Max iterations:',best_run_metrics['Max iterations:'])

best_run

Best Experiment Run:
-------------------------------------------
Best Run Id:  HD_b818826d-505b-45cb-b217-2efb3f91ff63_1
Accuracy: 0.8133333333333334
Regularization Strength: 0.8748876519670433
Max iterations: 25


Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-heart-failure-experiment,HD_b818826d-505b-45cb-b217-2efb3f91ff63_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
best_run.get_details()

{'runId': 'HD_b818826d-505b-45cb-b217-2efb3f91ff63_1',
 'target': 'hyperdrive-cpu-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-05-24T15:33:03.859661Z',
 'endTimeUtc': '2023-05-24T15:33:50.550075Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '0c2ced38-f2a9-4d73-905a-cc13213180d6',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.8748876519670433', '--max_iter', '25'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'hyperdrive-cpu-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'cred

In [19]:
best_run.get_metrics()

{'Regularization Strength:': 0.8748876519670433,
 'Max iterations:': 25,
 'Accuracy': 0.8133333333333334}

In [20]:
for primary_metric in best_run.get_metrics():
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

Regularization Strength: 0.8748876519670433
Max iterations: 25
Accuracy 0.8133333333333334


In [21]:
best_run.get_metrics(name='Accuracy')

{'Accuracy': 0.8133333333333334}

In [22]:
best_run.get_properties()

{'_azureml.ComputeTargetType': 'amlctrain',
 'ContentSnapshotId': '0c2ced38-f2a9-4d73-905a-cc13213180d6',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json'}

### Save the best model

In [25]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [26]:
best_run.download_file(name='outputs/model.joblib', output_file_path='./outputs')
best_model = best_run.register_model(model_name='best_run_hyperdrive', model_path='outputs/model.joblib')

Model(workspace=Workspace.create(name='quick-starts-ws-234282', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-234282'), name=best_run_hyperdrive, id=best_run_hyperdrive:2, version=2, tags={}, properties={})

In [27]:
from azureml.automl.core.shared import constants

# create inference folder
inference_folder = 'inference'
if inference_folder not in os.listdir():
    os.mkdir(inference_folder)

# Save the best model
##joblib.dump(best_model, filename = inference_folder + '/best_automl_model.joblib')
##joblib.dump(value=best_model, filename='output/best-automl.pkl')

# Save the best model, scoring script, and conda env files in inference folder
best_run.download_file('outputs/scoring_file_v_1_0_0.py', inference_folder + '/best_hyperdrive_score.py')
###best_run.download_file('outputs/model.pkl', inference_folder + '/best_hyperdrive_model.pkl')
best_run.download_file('outputs/model.joblib', inference_folder + '/best_hyperdrive_model.joblib')

best_run.download_file('outputs/conda_env_v_1_0_0.yml', inference_folder + '/hyperdrive_conda_env.yml')
##best_run.download_file(constants.CONDA_ENV_FILE_PATH, inference_folder + '/hyperdrive_conda_env.yml')

UserErrorException: UserErrorException:
	Message: File with path outputs/scoring_file_v_1_0_0.py was not found,
available files include: outputs/model.joblib,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/metrics_capability/metrics-capability.log,system_logs/snapshot_capability/snapshot-capability.log,user_logs/std_log.txt.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path outputs/scoring_file_v_1_0_0.py was not found,\navailable files include: outputs/model.joblib,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/metrics_capability/metrics-capability.log,system_logs/snapshot_capability/snapshot-capability.log,user_logs/std_log

### Save the environment

In [28]:
from azureml.core.environment import Environment

# get the list of environments
Environment.list(workspace=ws).keys()

dict_keys(['AzureML-PyTorch-1.3-CPU', 'AzureML-Dask-CPU', 'AzureML-Dask-GPU', 'AzureML-VowpalWabbit-8.8.0', 'AzureML-Triton', 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu', 'AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu', 'AzureML-sklearn-1.0-ubuntu20.04-py38-cpu', 'AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu', 'AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu', 'AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu', 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu', 'AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu', 'AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu', 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu', 'AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu', 'AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu', 'AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu', 'AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu', 'AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu', 'AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu'])

In [29]:
# save the environment
my_env = Environment.get(workspace=ws, name="AzureML-HyperDrive")
my_env.save_to_directory('env', overwrite=True)

my_env

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No environment exists for name: AzureML-HyperDrive, version: , label: ",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "eb20aae5776c624112e8a9693623454f",
    "request": "e585edf8bee9309f"
  },
  "environment": "westus2",
  "location": "westus2",
  "time": "2023-05-24T15:52:14.8960914+00:00",
  "componentName": "environment-management"
}

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [30]:
# Register the model
from azureml.core.resource_configuration import ResourceConfiguration

###model_name = best_run.properties['model_name']
model_name = best_run.properties['name']

local_file = inference_folder + '/best_automl_model.pkl'

run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace = ws,
                       model_name = model_name,                        # Name of the registered model in your workspace.
                       model_path = local_file,                        # Local file to upload and register as a model.
                       model_framework = Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version = sklearn.__version__,  # Version of scikit-learn used to create the model.
                       description = 'Best autoML model to predict motality caused by heart failure.',
                       tags={'area': 'heart-failure', 'type': 'classification'})

print('Model name:', model.name)
print('Model id:', model.id)
print('Model version:', model.version)

KeyError: 'model_name'

In [31]:
# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name="my_env", file_path=inference_folder + 'automl_conda_env.yml')
env
#### Siehe Oben my_env --> Ist es gleich?

inference_config = InferenceConfig(entry_script=inference_folder + '/best_automl_score.py', environment=env)

# display the environment file
with open(file_path=inference_folder + 'automl_conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)

SyntaxError: positional argument follows keyword argument (86904783.py, line 12)

In [32]:
# Model Deployment
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "heart-failure", 'type': "classification"},
                                                           description="Predict heart failure mortality using classification model",
                                                           auth_enabled=True,
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service = Model.deploy(workspace = ws, 
                           name = "aci-heart-failure-deploy", 
                           models = [model], 
                           inference_config = inference_config, 
                           deployment_config = aci_deployment_config, 
                           overwrite=True)

aci_service.wait_for_deployment(show_output=True)

NameError: name 'model' is not defined

In [33]:
# get the active api endpoint for scoring
print(f"Service State: {aci_service.state}\n")
print(f"Scoring URI:   {aci_service.scoring_uri}\n")
print(f"Swagger URI:   {aci_service.swagger_uri}\n")

NameError: name 'aci_service' is not defined

### Consuming the model
TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
# Send a request to the web service
import json
import requests

'''
input_data = json.dumps({
    "data": [
            [75.0, 0.0, 582.0, 0.0, 20.0, 1.0, 265000.0, 1.9, 130.0, 1.0, 0.0, 4.0],
            [80.0, 1.0, 123.0, 0.0, 35.0, 1.0, 388000.0, 9.4, 133.0, 1.0, 1.0, 10.0],
            [62.0, 0.0, 61.0, 1.0, 38.0, 1.0, 155000.0, 1.1, 143.0, 1.0, 1.0, 270.0],
            [50.0, 1.0, 111.0, 0.0, 20.0, 0.0, 210000.0, 1.9, 137.0, 1.0, 0.0, 7.0]
        ]
    })
'''

# 4 sets of data to score, so we get two results back
test_sample = test_df.sample(n=4)
labels = test_sample.pop('DEATH_EVENT')


# Convert to JSON string
input_data = json.dumps({"data": test_sample.to_dict(orient='records')})
with open("input_data.json", 'w') as _f:
    _f.write(input_data)

print(input_data)

response = requests.post(aci_service.scoring_uri, data=input_data, headers={'Content-Type':'application/json'})

{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 99, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 1, "platelets": 224000.0, "serum_creatinine": 2.5, "serum_sodium": 134, "sex": 1, "smoking": 0, "time": 162}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 95, "diabetes": 0, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 337000.0, "serum_creatinine": 1.0, "serum_sodium": 138, "sex": 1, "smoking": 1, "time": 146}, {"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 7702, "diabetes": 1, "ejection_fraction": 25, "high_blood_pressure": 1, "platelets": 390000.0, "serum_creatinine": 1.0, "serum_sodium": 139, "sex": 1, "smoking": 0, "time": 60}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 60, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 0, "platelets": 228000.0, "serum_creatinine": 1.2, "serum_sodium": 135, "sex": 1, "smoking": 1, "time": 90}]}


NameError: name 'aci_service' is not defined

In [35]:
print(f"Predictions from Service: {response.json()}\n")
print(f"Data Labels: {labels.tolist()}")

NameError: name 'response' is not defined

### Print the logs of the web service and delete the service
TODO: In the cell below, print the logs of the web service and delete the service

In [36]:
# Print the log of the webservice
print(aci_service.get_logs())

NameError: name 'aci_service' is not defined

In [37]:
# Delete the webservice, model, and shut down the compute cluster
aci_service.delete()
model.delete()
amlcompute_target.delete()

NameError: name 'aci_service' is not defined

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

